In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
# import MySQLdb as mdb
import numpy as np
import param
import panel as pn
import os

In [137]:
# config
pn.extension()


In [138]:
result_path = '../result/'
files = [f for f in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, f))]
# extract the date
dir_file = dict()
date_list = list()
for i in files:
    date_list.append(i[:8])
    dir_file[i[:8]] = i
    print(i[:8])
date_list.sort(reverse=True)
date_latest = date_list[0]
file_latest = os.path.join(result_path,dir_file[date_latest])
file_latest 

20210313
20210315
20210321
20210321
20210322
20210328
20210403
20210411
20210419
20210428
20210503
20210509
20210516
20210525
20210601
20210608
20210614
20210809
20210816
20210820
20210823
20211123


'../result/20211123_valuation_result_sp500.csv'

In [118]:
date_list.

['20210313',
 '20210315',
 '20210321',
 '20210321',
 '20210322',
 '20210328',
 '20210403',
 '20210411',
 '20210419',
 '20210428',
 '20210503',
 '20210509',
 '20210516',
 '20210525',
 '20210601',
 '20210608',
 '20210614',
 '20210809',
 '20210816',
 '20210820',
 '20210823',
 '20211123']

In [ ]:
# class ActionExample(param.Parameterized):
#     """
#     Demonstrates how to use param.Action to trigger an update.
#     """

#     action = param.Action(lambda x: x.param.trigger('action'), label='Click here!')
    
#     number = param.Integer(default=0)
        
#     @param.depends('action')
#     def get_number(self):
#         self.number += 1
#         return self.number

# action_example = ActionExample()
# # component = pn.Column(
# #     pn.Row(
# #         pn.Column(pn.panel(action_example, show_name=False, margin=0, widgets={"action": {"button_type": "primary"}, "number": {"disabled": True}}),
# #             '**Click the button** to trigger an update in the output.'),
# #         pn.panel(action_example.get_number, width=300), max_width=600)
# # )

# component = pn.Column(
#     pn.Row(
#         pn.Column(pn.panel(action_example, show_name=False, margin=0, widgets={"action": {"button_type": "primary"}})),
#         pn.panel(action_example.get_number, width=300), max_width=600)
# )
# component

Column(sizing_mode='stretch_width')
    [0] Row(max_width=600, sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] Column(name='ActionExample', sizing_mode='stretch_width', width=300)
                [0] Button(button_type='primary', name='Click here!', sizing_mode='stretch_width')
                [1] IntInput(name='Number', sizing_mode='stretch_width', value=1)
        [1] ParamMethod(method, sizing_mode='stretch_width', width=300)

In [139]:
# class ActionLoadValuation(param.Parameterized):
#     """
#     click the button to load valuation results
#     """
#     action = param.Action(lambda x: x.param.trigger('action'), label='Load valuation results')
    
#     df = param.DataFrame()
    
#     @param.depends('action')
#     def load_valuation_dataframe(self):
#         #file = '../result/20210809_valuation_result_sp500.csv'
#         result_path = '../result/'
#         files = [f for f in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, f))]
#         # extract the date
#         dir_file = dict()
#         date_list = list()
#         for i in files:
#             date_list.append(i[:8])
#             dir_file[i[:8]] = i
#             #print(i[:8])
#         date_list.sort(reverse=True)
#         date_latest = date_list[0]
#         file_latest = os.path.join(result_path,dir_file[date_latest])
#         self.df = pd.read_csv(file_latest)
#         return self.df

# action_load_valuation = ActionLoadValuation()
# component_load_valuation = pn.Column(
#     pn.Row(
#         pn.Column(pn.panel(action_load_valuation, show_name=False, margin=0, widgets={"action": {"button_type": "primary"}})),
#         pn.panel(action_load_valuation.load_valuation_dataframe, width=300), max_width=600)
# )
# # component_load_valuation

In [149]:
button_load_valuation = pn.widgets.Button(name='Load valuation results', button_type='primary')
text = pn.widgets.TextInput(value='Ready')
df_valuation = pn.widgets.DataFrame()

def button_load_valuation_process(event):
    #file = '../result/20210809_valuation_result_sp500.csv'
    result_path = '../result/'
    files = [f for f in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, f))]
    # extract the date
    dir_file = dict()
    date_list = list()
    for i in files:
        date_list.append(i[:8])
        dir_file[i[:8]] = i
        #print(i[:8])
    date_list.sort(reverse=True)
    date_latest = date_list[0]
    file_latest = os.path.join(result_path,dir_file[date_latest])
    text.value = 'Valuation at {}'.format(file_latest[-35:-27])
    df_valuation.value = pd.read_csv(file_latest)
    
button_load_valuation.on_click(button_load_valuation_process)
pn.Row(button_load_valuation, text)

Row(sizing_mode='stretch_width')
    [0] Button(button_type='primary', name='Load valuation results', sizing_mode='stretch_width')
    [1] TextInput(sizing_mode='stretch_width', value='Ready')

In [150]:
df_valuation

DataFrame(sizing_mode='stretch_width', value=    ticker  current_price ...)

### method 1

In [4]:
# create function dropdown
def func_valuation(ticker='UAL'):
    return df_valuation.iloc[np.where(df_valuation.ticker.values==ticker)]
    

import panel as pn
pn.extension()
pn.interact(func_valuation)

kw = dict(ticker=sorted(list(df_valuation.ticker.values)))
i = pn.interact(func_valuation, **kw)
i.pprint()

text = f"<br>\n# Valuation on {file[10:18]} \nSelect the ticker"
p = pn.Row(pn.Column(text, i[0]), i[1][0])
p


Column
    [0] Column
        [0] Select(name='ticker', options=['A', 'AAL', 'AAP', ...], value='UAL')
    [1] Row
        [0] DataFrame(DataFrame, name='interactive00108')


Row
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Select(name='ticker', options=['A', 'AAL', 'AAP', ...], value='UAL')
    [1] DataFrame(DataFrame, name='interactive00108')

### method 2

In [5]:
# create widget
tickers=sorted(list(df_valuation.ticker.values))
select = pn.widgets.Select(name='ticker', options=tickers)

# create function manual input
def func_valuation(ticker='UAL'):
    return df_valuation.iloc[np.where(df_valuation.ticker.values==ticker)]

# create interaction between widgt and function
pn.interact(func_valuation, ticker=select)

Column
    [0] Column
        [0] Select(name='ticker', options=['A', 'AAL', 'AAP', ...], value='A')
    [1] Row
        [0] DataFrame(DataFrame, name='interactive00123')